In [1]:
import pandas as pd 
import numpy as np
import os
from sklearn.utils import resample

In [2]:
# check see what's data looks like 
df_demo=pd.read_csv('../data/bending1/dataset1.csv',skiprows=4)

In [3]:
df_demo.head(3), df_demo.describe()

(   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
 0                0      39.25       0.43      22.75       0.43      33.75   
 1              250      39.25       0.43      23.00       0.00      33.00   
 2              500      39.25       0.43      23.25       0.43      33.00   
 
    var_rss23  
 0        1.3  
 1        0.0  
 2        0.0  ,
        # Columns: time   avg_rss12   var_rss12   avg_rss13   var_rss13  \
 count       480.000000  480.000000  480.000000  480.000000  480.000000   
 mean      59875.000000   40.624792    0.358604   19.040937    0.832542   
 std       34677.081769    1.476967    0.322605    4.462952    0.965659   
 min           0.000000   37.250000    0.000000    4.000000    0.000000   
 25%       29937.500000   39.250000    0.000000   16.000000    0.000000   
 50%       59875.000000   40.500000    0.430000   19.250000    0.500000   
 75%       89812.500000   42.000000    0.500000   23.250000    1.120000   
 max      119750.0000

#### 1c


#### (i) median is widely used time feature as it tells the center of a dataset without being affacted by outliers

In [6]:
# (ii)
df_all = pd.DataFrame()

df_act = {'bending1': [2, 7], 'bending2': [2, 6],  'cycling': [3, 15],'lying': [3, 15],'sitting': [3, 15],'standing': [3, 15],'walking': [3, 15]}

# function extract all features from df 
def extract_features(df, col):
    return {
        f'{col}_min': df[col].min(),
        f'{col}_max': df[col].max(),
        f'{col}_mean': df[col].mean(),
        f'{col}_median': df[col].median(),
        f'{col}_std': df[col].std(),
        f'{col}_q1': df[col].quantile(0.25),
        f'{col}_q3': df[col].quantile(0.75)
    }

# loop all files 
for act in df_act.keys():
    test_num = df_act[act][0] 
    train_num = df_act[act][1]  
    
    # for each activity loop all data
    for i in range(1, train_num + 1):
        file_path = f'../data/{act}/dataset{i}.csv'
        
        # check data
        try:
            df = pd.read_csv(file_path, skiprows=4)
        except:
            print(f"File {file_path} .")
            continue
        
        df = df.rename(columns={'# Columns: time': 'time'})
        df = df.drop(columns=['time']) 

        features = {}
        for column in df.columns:
            features.update(extract_features(df, column))

        # add identifier for test or train datas
        features['test'] = 1 if i <= test_num else 0
        features['activity'] = act
        features['file_number'] = i

        df_features = pd.DataFrame([features])
        df_all = pd.concat([df_all, df_features], ignore_index=True)

# check 
print(df_all.shape, list(df_all))
df_all.head(3)

(88, 45) ['avg_rss12_min', 'avg_rss12_max', 'avg_rss12_mean', 'avg_rss12_median', 'avg_rss12_std', 'avg_rss12_q1', 'avg_rss12_q3', 'var_rss12_min', 'var_rss12_max', 'var_rss12_mean', 'var_rss12_median', 'var_rss12_std', 'var_rss12_q1', 'var_rss12_q3', 'avg_rss13_min', 'avg_rss13_max', 'avg_rss13_mean', 'avg_rss13_median', 'avg_rss13_std', 'avg_rss13_q1', 'avg_rss13_q3', 'var_rss13_min', 'var_rss13_max', 'var_rss13_mean', 'var_rss13_median', 'var_rss13_std', 'var_rss13_q1', 'var_rss13_q3', 'avg_rss23_min', 'avg_rss23_max', 'avg_rss23_mean', 'avg_rss23_median', 'avg_rss23_std', 'avg_rss23_q1', 'avg_rss23_q3', 'var_rss23_min', 'var_rss23_max', 'var_rss23_mean', 'var_rss23_median', 'var_rss23_std', 'var_rss23_q1', 'var_rss23_q3', 'test', 'activity', 'file_number']


,avg_rss12_min,avg_rss12_max,avg_rss12_mean,avg_rss12_median,avg_rss12_std,avg_rss12_q1,avg_rss12_q3,var_rss12_min,var_rss12_max,var_rss12_mean,...,var_rss23_min,var_rss23_max,var_rss23_mean,var_rss23_median,var_rss23_std,var_rss23_q1,var_rss23_q3,test,activity,file_number
0,37.25,45.00,40.624792,40.50,1.476967,39.25,42.00,0.0,1.30,0.358604,...,0.0,1.92,0.570583,0.43,0.582915,0.0,1.30,1,bending1,1
1,38.00,45.67,42.812812,42.50,1.435550,42.00,43.67,0.0,1.22,0.372437,...,0.0,3.11,0.571083,0.43,0.601010,0.0,1.30,1,bending1,2
2,35.00,47.40,43.954500,44.33,1.558835,43.00,45.00,0.0,1.70,0.426250,...,0.0,1.79,0.493292,0.43,0.513506,0.0,0.94,0,bending1,3


In [7]:
# check null
df_all.isnull().sum().sum()

0

#### 1cii

In [9]:
df_all_cp = df_all.drop(columns=['test','activity','file_number']) 
feature_columns = list(df_all_cp)
results = {}

# calculate sd and ci 
def bootstrap_ci(data, num_bootstrap=1000, ci=90):

    sds = []
    for i in range(num_bootstrap):
        sample = resample(data)
        sds.append(np.std(sample,ddof=1))
    
    lower_bound = np.percentile(sds,(100-ci)/2)
    upper_bound = np.percentile(sds,100-(100-ci)/2)
    
    return lower_bound, upper_bound

for col in feature_columns:
    std_value = np.std(df_all_cp[col])
    lower_ci, upper_ci = bootstrap_ci(df_all_cp[col])

    results[col] = {'sd': std_value,'ci_lower': lower_ci,'ci_upper': upper_ci}

df_results = pd.DataFrame(results).T
df_results.columns = ['SD', 'CI Low', 'CI Upper']
df_results.describe(), df_results.isnull().sum(), df_results.head(2)

(              SD     CI Low   CI Upper
 count  42.000000  42.000000  42.000000
 mean    2.982628   2.552114   3.376373
 std     2.323120   1.944375   2.689070
 min     0.000000   0.000000   0.000000
 25%     1.034364   0.870527   1.162853
 50%     2.142170   1.938465   2.324724
 75%     5.090702   4.350012   5.742454
 max     9.515445   8.265395  10.720608,
 SD          0
 CI Low      0
 CI Upper    0
 dtype: int64,
                      SD    CI Low   CI Upper
 avg_rss12_min  9.515445  8.265395  10.720608
 avg_rss12_max  4.369322  3.389046   5.296506)

#### 1ciV: the three most important time-domain features are median, q1, and q3. This captrue majority of the data and avoid problem with outliers. 

In [19]:
df_results

,SD,CI Low,CI Upper
avg_rss12_min,9.515445,8.265395,10.720608
avg_rss12_max,4.369322,3.389046,5.296506
avg_rss12_mean,5.305314,4.720318,5.872720
avg_rss12_median,5.409056,4.817767,6.001591
avg_rss12_std,1.762056,1.557597,1.946415
avg_rss12_q1,6.118526,5.568565,6.655448
avg_rss12_q3,5.109643,4.354950,5.838787
var_rss12_min,0.000000,0.000000,0.000000
var_rss12_max,5.033882,4.643629,5.392854
var_rss12_mean,1.565194,1.405429,1.707313
